In [6]:
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%pip install torchvision
import torchvision
import torch
import os 

Note: you may need to restart the kernel to use updated packages.


In [16]:
DATASET_PATH  = r'C:\Users\13474\OneDrive\Documents\CS501\projectrepo\google_maps_queries\output'
# TEST_DIR = 'google_maps_queries/output'
IMAGE_SIZE    = (300,300)
NUM_CLASSES   = 3
BATCH_SIZE    = 10  
LEARNING_RATE = 0.001 

In [20]:
dataset = torchvision.datasets.ImageFolder(DATASET_PATH)
test_size = 0.2 * len(dataset)
# print(test_size)
test_set = torch.utils.data.Subset(dataset, range(int(test_size)))
train_set = torch.utils.data.Subset(dataset, range(int(test_size), len(dataset)))
# indexes = shuffle(range(len(dataset)))
# indexes_train = indexes[:int(len(dataset)*0.9)]
# indexes_test = indexes[int(len(dataset)*0.9):]


# train_dataset = image_dataset_from_directory(directory= DATASET_PATH, label_mode='categorical', class_names = ['OCCUPIED_STRUCTURE', 'VACANT_LOT','VACANT_STRUCTURE'])
# test_dataset = image_dataset_from_directory(directory= TEST_DIR, label_mode='categorical',class_names = ['OCCUPIED_STRUCTURE', 'VACANT_LOT','VACANT_STRUCTURE'])

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,featurewise_center = True,
                                   featurewise_std_normalization = True,width_shift_range=0.2,
                                   height_shift_range=0.2,shear_range=0.25,zoom_range=0.1,
                                   zca_whitening = True,channel_shift_range = 20,
                                   horizontal_flip = True,vertical_flip = True,
                                   validation_split = 0.2,fill_mode='constant')

train_batches = train_datagen.flow_from_directory(DATASET_PATH,target_size=IMAGE_SIZE,
                                                  shuffle=True,batch_size=BATCH_SIZE,
                                                  subset = "training",seed=42,
                                                  class_mode="categorical")

valid_batches = train_datagen.flow_from_directory(DATASET_PATH,target_size=IMAGE_SIZE,
                                                  shuffle=True,batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,class_mode="categorical")

model = Sequential()

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(300,300, 3))

for layer in base_model.layers:
  layer.trainable=False

model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

C:\Users\13474\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:341: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


Found 14 images belonging to 3 classes.
Found 2 images belonging to 3 classes.


AttributeError: 'Subset' object has no attribute 'numpy'

In [28]:
import numpy as np 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit((train_set), batch_size=32, epochs=10)

model.evaluate(np.array(test_set), batch_size=32)

ValueError: Failed to find data adapter that can handle input: <class 'torch.utils.data.dataset.Subset'>, <class 'NoneType'>

In [ ]:
model.save('model')